In [1]:
! pip install yfinance langchain_pinecone openai python-dotenv langchain-community sentence_transformers

In [2]:
from langchain_pinecone import PineconeVectorStore
from openai import OpenAI
import dotenv
import json
import yfinance as yf
import concurrent.futures
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import numpy as np
import requests
import os
from dotenv import load_dotenv
load_dotenv()

False

In [3]:
def get_stock_info(symbol: str) -> dict:
    """
    Retrieves and formats detailed information about a stock from Yahoo Finance.

    Args:
        symbol (str): The stock ticker symbol to look up.

    Returns:
        dict: A dictionary containing detailed stock information, including ticker, name,
              business summary, city, state, country, industry, and sector.
    """
    headers = {
        'Authorization': f'Bearer {os.getenv("YAHOO_ACCESS_TOKEN")}'
    }
    session = requests.Session()
    session.headers.update(headers)

    data = yf.Ticker(symbol, session=session)
    # data = yf.Ticker(symbol)
    stock_info = data.info

    properties = {
        "Ticker": stock_info.get('symbol', 'Information not available'),
        'Name': stock_info.get('longName', 'Information not available'),
        'Business Summary': stock_info.get('longBusinessSummary'),
        'City': stock_info.get('city', 'Information not available'),
        'State': stock_info.get('state', 'Information not available'),
        'Country': stock_info.get('country', 'Information not available'),
        'Industry': stock_info.get('industry', 'Information not available'),
        'Sector': stock_info.get('sector', 'Information not available'),
        'Market Cap': stock_info.get('marketCap', 'Information not available'),
        'Volume': stock_info.get('volume', 'Information not available'),
    }

    return properties
    # return stock_info

In [4]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    """
    Generates embeddings for the given text using a specified Hugging Face model.

    Args:
        text (str): The input text to generate embeddings for.
        model_name (str): The name of the Hugging Face model to use.
                          Defaults to "sentence-transformers/all-mpnet-base-v2".

    Returns:
        np.ndarray: The generated embeddings as a NumPy array.
    """
    model = SentenceTransformer(model_name)
    return model.encode(text)


def cosine_similarity_between_sentences(sentence1, sentence2):
    """
    Calculates the cosine similarity between two sentences.

    Args:
        sentence1 (str): The first sentence for similarity comparison.
        sentence2 (str): The second sentence for similarity comparison.

    Returns:
        float: The cosine similarity score between the two sentences,
               ranging from -1 (completely opposite) to 1 (identical).

    Notes:
        Prints the similarity score to the console in a formatted string.
    """
    # Get embeddings for both sentences
    embedding1 = np.array(get_huggingface_embeddings(sentence1))
    embedding2 = np.array(get_huggingface_embeddings(sentence2))

    # Reshape embeddings for cosine_similarity function
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    similarity_score = similarity[0][0]
    print(f"Cosine similarity between the two sentences: {similarity_score:.4f}")
    return similarity_score


# Example usage
sentence1 = "I like walking to the park"
sentence2 = "I like running to the office"

similarity = cosine_similarity_between_sentences(sentence1, sentence2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Cosine similarity between the two sentences: 0.6133


In [5]:
def get_company_tickers():
    """
    Downloads and parses the Stock ticker symbols from the GitHub-hosted SEC company tickers JSON file.

    Returns:
        dict: A dictionary containing company tickers and related information.

    Notes:
        The data is sourced from the official SEC website via a GitHub repository:
        https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json
    """
    # URL to fetch the raw JSON file from GitHub
    url = "https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json"

    # Making a GET request to the URL
    response = requests.get(url)

    # Checking if the request was successful
    if response.status_code == 200:
        # Parse the JSON content directly
        company_tickers = json.loads(response.content.decode('utf-8'))

        # Optionally save the content to a local file for future use
        with open("company_tickers.json", "w", encoding="utf-8") as file:
            json.dump(company_tickers, file, indent=4)

        print("File downloaded successfully and saved as 'company_tickers.json'")
        return company_tickers
    else:
        print(f"Failed to download file. Status code: {response.status_code}")
        return None

company_tickers = get_company_tickers()

File downloaded successfully and saved as 'company_tickers.json'


In [6]:
company_tickers

{'0': {'cik_str': 1045810, 'ticker': 'NVDA', 'title': 'NVIDIA CORP'},
 '1': {'cik_str': 320193, 'ticker': 'AAPL', 'title': 'Apple Inc.'},
 '2': {'cik_str': 789019, 'ticker': 'MSFT', 'title': 'MICROSOFT CORP'},
 '3': {'cik_str': 1018724, 'ticker': 'AMZN', 'title': 'AMAZON COM INC'},
 '4': {'cik_str': 1652044, 'ticker': 'GOOGL', 'title': 'Alphabet Inc.'},
 '5': {'cik_str': 1326801, 'ticker': 'META', 'title': 'Meta Platforms, Inc.'},
 '6': {'cik_str': 1318605, 'ticker': 'TSLA', 'title': 'Tesla, Inc.'},
 '7': {'cik_str': 1067983,
  'ticker': 'BRK-B',
  'title': 'BERKSHIRE HATHAWAY INC'},
 '8': {'cik_str': 1046179,
  'ticker': 'TSM',
  'title': 'TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD'},
 '9': {'cik_str': 1730168, 'ticker': 'AVGO', 'title': 'Broadcom Inc.'},
 '10': {'cik_str': 59478, 'ticker': 'LLY', 'title': 'ELI LILLY & Co'},
 '11': {'cik_str': 19617, 'ticker': 'JPM', 'title': 'JPMORGAN CHASE & CO'},
 '12': {'cik_str': 104169, 'ticker': 'WMT', 'title': 'Walmart Inc.'},
 '13': {'cik_str'

In [7]:
len(company_tickers)

9998

Get all the Stocks in the Stock Market

In [18]:
def get_company_tickers():
    """
    Downloads and parses the Stock ticker symbols from the GitHub-hosted SEC company tickers JSON file.

    Returns:
        dict: A dictionary containing company tickers and related information.

    Notes:
        The data is sourced from the official SEC website via a GitHub repository:
        https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json
    """
    # URL to fetch the raw JSON file from GitHub
    url = "https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json"

    # Making a GET request to the URL
    response = requests.get(url)

    # Checking if the request was successful
    if response.status_code == 200:
        # Parse the JSON content directly
        company_tickers = json.loads(response.content.decode('utf-8'))

        # Optionally save the content to a local file for future use
        with open("company_tickers.json", "w", encoding="utf-8") as file:
            json.dump(company_tickers, file, indent=4)

        print("File downloaded successfully and saved as 'company_tickers.json'")
        return company_tickers
    else:
        print(f"Failed to download file. Status code: {response.status_code}")
        return None

company_tickers = get_company_tickers()

File downloaded successfully and saved as 'company_tickers.json'


In [19]:
def get_stock_info(symbol: str) -> dict:
    data = yf.Ticker(symbol)
    stock_info = data.info

    properties = {
        "Ticker": stock_info.get("symbol", "Information not available"),
        "Name": stock_info.get("longName", "Information not available"),
        "Business Summary": stock_info.get(
            "longBusinessSummary", "Information not available"
        ),
        "City": stock_info.get("city", "Information not available"),
        "State": stock_info.get("state", "Information not available"),
        "Country": stock_info.get("country", "Information not available"),
        "Industry": stock_info.get("industry", "Information not available"),
        "Sector": stock_info.get("sector", "Information not available"),
    }

    return properties

In [20]:
# Initialize tracking lists
successful_tickers = []
unsuccessful_tickers = []

# Load existing successful/unsuccessful tickers
try:
    with open('successful_tickers.txt', 'r') as f:
        successful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(successful_tickers)} successful tickers")
except FileNotFoundError:
    print("No existing successful tickers file found")

try:
    with open('unsuccessful_tickers.txt', 'r') as f:
        unsuccessful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(unsuccessful_tickers)} unsuccessful tickers")
except FileNotFoundError:
    print("No existing unsuccessful tickers file found")

Loaded 3900 successful tickers
Loaded 9 unsuccessful tickers


In [21]:
import os
print(f"Number of CPU cores: {os.cpu_count()}")

Number of CPU cores: 8


Inserting Stocks into Pinecone

In [22]:
def process_stock(stock_ticker: str) -> str:

    if stock_ticker in successful_tickers:
        return f"Already processed {stock_ticker}"

    try:
        # Get and store stock data
        stock_data = get_stock_info(stock_ticker)
        if stock_data['Business Summary'] is None:
            stock_data['Business Summary'] = "No business summary available"
        stock_description = stock_data['Business Summary']

        vectorstore_from_texts = PineconeVectorStore.from_documents(
            documents=[Document(page_content=stock_description, metadata=stock_data)],
            embedding=hf_embeddings,
            index_name=index_name,
            namespace=namespace
        )

        # Track success
        with open('successful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        successful_tickers.append(stock_ticker)

        return f"Processed {stock_ticker} successfully"

    except Exception as e:
        # Track failure
        with open('unsuccessful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        unsuccessful_tickers.append(stock_ticker)

        return f"ERROR processing {stock_ticker}: {e}"

def parallel_process_stocks(tickers: list, batch_size=35, max_workers: int = 5) -> None:
    for i in range(0, len(tickers), batch_size):
        print(f"\nProcessing batch {i//batch_size + 1}")
        batch = tickers[i:i + batch_size]

        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_ticker = {
            executor.submit(process_stock, ticker): ticker
            for ticker in batch
        }

        for future in concurrent.futures.as_completed(future_to_ticker):
            ticker = future_to_ticker[future]
            try:
                result = future.result()
                print(result)

            except Exception as exc:
                print(f'ERROR processing {ticker}: {exc}')
                continue

# Prepare your tickers
tickers_to_process = [company_tickers[num]['ticker'] for num in company_tickers.keys()]

# Process them
parallel_process_stocks(tickers_to_process, max_workers=7)


Processing batch 1
Already processed CVX
Already processed AAPL
Already processed CRM
Already processed BAC
Already processed UNH
Already processed MA
Already processed NVO
Already processed SPY
Already processed ABBV
Already processed JPM
Already processed AVGO
Already processed SAP
Already processed GOOGL
Already processed TSM
Already processed LLY
Already processed BRK-B
Already processed MSFT
Already processed NVDA
Already processed PG
Already processed AMZN
Already processed NFLX
Already processed KO
Already processed ASML
Already processed HD
Already processed RCIT
Already processed ORCL
Already processed JNJ
Already processed V
Already processed XOM
Already processed WFC
Already processed WMT
Already processed META
Already processed COST
Already processed TMUS
Already processed TSLA

Processing batch 2
Already processed AZN
Already processed PEP
Already processed TM
Already processed AXP
Already processed SHEL
Already processed ADBE
Already processed AMD
Already processed GE
Al

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/IGI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=IGI&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:401 Client Error: Unauthorized for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ELLO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ELLO&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:401 Client Error: Unauthorized for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CRBU?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CRBU&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSu

ERROR processing CAMP: You haven't specified an Api-Key.
ERROR processing SKGR: You haven't specified an Api-Key.
ERROR processing NRO: You haven't specified an Api-Key.
ERROR processing CRBU: You haven't specified an Api-Key.
ERROR processing CLLS: You haven't specified an Api-Key.
ERROR processing PLCE: You haven't specified an Api-Key.
ERROR processing AOAO: You haven't specified an Api-Key.
ERROR processing HRTX: You haven't specified an Api-Key.
ERROR processing PAYS: You haven't specified an Api-Key.
ERROR processing PROP: You haven't specified an Api-Key.
ERROR processing ICTSF: You haven't specified an Api-Key.
ERROR processing VSTM: You haven't specified an Api-Key.
ERROR processing EM: Expecting value: line 1 column 1 (char 0)
ERROR processing FET: You haven't specified an Api-Key.
ERROR processing ISBA: You haven't specified an Api-Key.
ERROR processing INFU: You haven't specified an Api-Key.
ERROR processing RDCM: You haven't specified an Api-Key.
ERROR processing PERF: You

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/RMNI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=RMNI&crumb=3h7kDP8wDf2
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GALT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=GALT&crumb=3h7kDP8wDf2
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AVNW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AVNW&crumb=3h7kDP8wDf2
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SPKL?modules=financialDa

ERROR processing SMHI: You haven't specified an Api-Key.
ERROR processing JRVR: You haven't specified an Api-Key.
ERROR processing ACTU: You haven't specified an Api-Key.
ERROR processing FLC: You haven't specified an Api-Key.
ERROR processing GLDG: You haven't specified an Api-Key.
ERROR processing ARAY: You haven't specified an Api-Key.
ERROR processing ATOM: You haven't specified an Api-Key.
ERROR processing AVD: You haven't specified an Api-Key.
ERROR processing HGLB: You haven't specified an Api-Key.
ERROR processing GDO: You haven't specified an Api-Key.
ERROR processing GLSI: You haven't specified an Api-Key.
ERROR processing RMNI: You haven't specified an Api-Key.
ERROR processing IDE: You haven't specified an Api-Key.
ERROR processing AXR: You haven't specified an Api-Key.
ERROR processing SPKL: You haven't specified an Api-Key.
ERROR processing BRKH: You haven't specified an Api-Key.
ERROR processing CDLX: You haven't specified an Api-Key.
ERROR processing LFVN: You haven't s

...

ERROR processing WSKEF: Expecting value: line 1 column 1 (char 0)
ERROR processing YELLQ: Expecting value: line 1 column 1 (char 0)
ERROR processing YQAI: Expecting value: line 1 column 1 (char 0)
ERROR processing UONEK: Expecting value: line 1 column 1 (char 0)
ERROR processing MDXL: Expecting value: line 1 column 1 (char 0)
ERROR processing NBST: Expecting value: line 1 column 1 (char 0)
ERROR processing XTXXF: Expecting value: line 1 column 1 (char 0)
ERROR processing RSCI: Expecting value: line 1 column 1 (char 0)
ERROR processing ENDI: Expecting value: line 1 column 1 (char 0)
ERROR processing INFT: Expecting value: line 1 column 1 (char 0)
ERROR processing NZEOF: Expecting value: line 1 column 1 (char 0)
ERROR processing GBNY: Expecting value: line 1 column 1 (char 0)
ERROR processing SMTSF: Expecting value: line 1 column 1 (char 0)
ERROR processing BIOE: Expecting value: line 1 column 1 (char 0)
ERROR processing AVLNF: Expecting value: line 1 column 1 (char 0)
ERROR processing F

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MBBC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MBBC&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FIGI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FIGI&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BQST?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BQST&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/fin

ERROR processing ARRKF: Expecting value: line 1 column 1 (char 0)
ERROR processing TETAB: Expecting value: line 1 column 1 (char 0)
ERROR processing SGLDF: Expecting value: line 1 column 1 (char 0)
ERROR processing GLACU: Expecting value: line 1 column 1 (char 0)
ERROR processing AFGC: Expecting value: line 1 column 1 (char 0)
ERROR processing SEAC: Expecting value: line 1 column 1 (char 0)
ERROR processing NPLS: Expecting value: line 1 column 1 (char 0)
ERROR processing FNCH: Expecting value: line 1 column 1 (char 0)
ERROR processing BRCNF: Expecting value: line 1 column 1 (char 0)
ERROR processing CSSI: Expecting value: line 1 column 1 (char 0)
ERROR processing PWCO: Expecting value: line 1 column 1 (char 0)
ERROR processing SFWJ: Expecting value: line 1 column 1 (char 0)
ERROR processing SPTY: Expecting value: line 1 column 1 (char 0)
ERROR processing WHLRD: Expecting value: line 1 column 1 (char 0)
ERROR processing CATG: Expecting value: line 1 column 1 (char 0)
ERROR processing SK

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BNSOF?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BNSOF&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LNDZF?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LNDZF&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TVE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TVE&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/f

KeyboardInterrupt: 

In [ ]:
# Remove tickers that were successfully processed from the unsuccessful_tickers.txt file
for ticker in unsuccessful_tickers:
  if ticker in successful_tickers:
    print(f"Removing {ticker} from unsuccessful_tickers.txt")
    with open('unsuccessful_tickers.txt', 'r') as f:
        lines = f.readlines()
    with open('unsuccessful_tickers.txt', 'w') as f:
        for line in lines:
            if line.strip() != ticker:
                f.write(line)

successful_tickers = []
unsuccessful_tickers = []

# Load existing successful/unsuccessful tickers
try:
    with open('successful_tickers.txt', 'r') as f:
        successful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(successful_tickers)} successful tickers")
except FileNotFoundError:
    print("No existing successful tickers file found")

try:
    with open('unsuccessful_tickers.txt', 'r') as f:
        unsuccessful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(unsuccessful_tickers)} unsuccessful tickers")
except FileNotFoundError:
    print("No existing unsuccessful tickers file found")

In [ ]:
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)

pinecone_index = pc.Index(index_name)

In [ ]:
def run_streamlit():
  os.system("streamlit run app.py --server.port 8501")

In [23]:
%%writefile app.py
from langchain_pinecone import PineconeVectorStore
from openai import OpenAI
import json
import yfinance as yf
import concurrent.futures
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import numpy as np
import requests
import os
import streamlit as st
from dotenv import load_dotenv
load_dotenv()

pinecone_api_key = os.getenv('PINECONE_API_KEY')

index_name = "stocks"
namespace = "stock-descriptions"

hf_embeddings = HuggingFaceEmbeddings()
vectorstore = PineconeVectorStore(index_name=index_name, embedding=hf_embeddings)


pc = Pinecone(api_key=pinecone_api_key)

pinecone_index = pc.Index(index_name)

groq_api_key = os.getenv('GROQ_API_KEY')

client = OpenAI(
  base_url="https://api.groq.com/openai/v1",
  api_key=groq_api_key
)

def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

def get_stock_info_all(symbol: str) -> dict:
    headers = {
        'Authorization': f'Bearer {os.getenv("YAHOO_ACCESS_TOKEN")}'
    }
    session = requests.Session()
    session.headers.update(headers)

    data = yf.Ticker(symbol, session=session)

    stock_info = data.info

    return stock_info

def format_filter_conditions(filter_conditions):
    if not filter_conditions:
        return ""

    formatted_filters = []

    for key, value in filter_conditions.items():
        if isinstance(value, dict):

            for op, val in value.items():
                operator_map = {
                    "$gte": "greater than or equal to",
                    "$lte": "less than or equal to",
                    "$gt": "greater than",
                    "$lt": "less than",
                    "$eq": "equals",
                    "$in": "in",
                }
                op_text = operator_map.get(op, op)
                formatted_filters.append(f"{key} is {op_text} {val}")
        else:

            formatted_filters.append(f"{key}: {value}")

    return ", ".join(formatted_filters)


def HandleQuery(query, filter_conditions):
    filter_conditions_string = format_filter_conditions(filter_conditions)

    raw_query_embedding = get_huggingface_embeddings(query)

    top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=10, include_metadata=True, namespace=namespace,filter=filter_conditions if filter_conditions else None)

    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query + filter_conditions_string

    system_prompt = f"""You are an expert at providing answers about stocks. Please answer my question provided.

    When giving your response, please do not mention the context provided to you or the query.

    Please provide a detailed answer to the question.

    Please provide all of the answers that you receive from the context provided.

    Please provide the answers from most relevant to least relevant.

    Please provide the answer in a markdown format.

    Please be consistent in the markdown format for all of your answers.

    If no question is provided, please provide a list of all of the stocks that match the filters and their information.
    """

    llm_response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    response = llm_response.choices[0].message.content
    return response

st.title('Stock Analysis')
st.warning("Keep in mind that more detailed your query and filters are, the more relevant and accurate the results will be.")

st.write("You can use the following filters to narrow down the results:")
st.write("Market Cap and Volume will return results that are greater than or equal to the value you enter.")
industry = st.text_input('Industry:',)
sector = st.text_input('Sector:',)
market_cap = st.number_input(
    'Market Cap:',
    min_value=0,
    max_value=1000000,
    step=1
)
volume = st.number_input(
    'Volume:',
    min_value=0,
    max_value=1000000,
    step=1
)

st.write("Ask general questions about stocks:")
query = st.text_input('Ask About Stocks:',)

filter = {
    "industry": industry,
    "sector": sector,
    "marketCap": {"$gte": market_cap},
    "volume": {"$gte": volume}
}

if st.button('Get Stock Info'):
    st.write(f'Getting info for {query}...')
    response = HandleQuery(query, filter)

    st.write("### Response:")
    st.write(response)


    st.markdown("---")

    if not response:
        st.error("No information found for this query.")

Writing app.py
